In [ ]:
import pandas as pd
import re
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from contractions import fix

In [2]:
raw_scraper_results = pd.read_csv("../test-data/Bluesky Scraper Results - Selected Data Points.csv")
print(len(raw_scraper_results))
raw_scraper_results.head()

211


,X (Post Content),"y (label) [1 = deplatform positive, 0 = not deplatformed]"
0,If you want to message me on telegram my gener...,1
1,Please feel free to send me a direct message o...,1
2,I’m doing quite well how about you? Would you ...,1
3,Send me a message on Donald Trump Mega fan pa...,1
4,I am taking MFF delivery magnet and pin commis...,1


In [ ]:
def preprocess(data: pd.DataFrame):
    out = data.copy(deep=True)
    out["text"] = out["X (Post Content)"].str.lower()
    out["text"] = out["text"].apply(lambda x: fix(x)) # Expand contractions
    out["text"] = out["text"].apply(lambda x: re.sub(r'@[\w_]+', '<MENTION>', x)) # Replace mention with MENTION token.
    out["text"] = out["text"].apply(lambda x: re.sub(r'http\S+|www\.\S+', '<URL>', x)) # Replace url with URL token.
    out["text"] = out["text"].apply(lambda x: re.sub(
        r'(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})(?:[-. ]*(?:ext|x|ext\.)\s*(\d+))?', '<PHONE_NUMBER>', x
        )) # Replace phone number with PHONE_NUMBER token.
    out["text"] = out["text"].apply(lambda x: re.sub(r'\b\d+\b', '<NUM>', x)) # Replace all other numbers with NUM token.
    out["text"] = out["text"].apply(lambda x: re.sub(r't\.me/[a-zA-Z0-9_]+', '<TELEGRAM_HANDLE>', x)) # Replace telegram handle with TELEGRAM_HANDLE token.

    def replace_emoji(text):
        return re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+'
                    , '<EMOJI>', text)
    
    out["text"] = out["text"].apply(replace_emoji)
    out["tokens"] = out["text"].apply(word_tokenize)
    out["contains_exclamation"] = out["text"].str.contains("!").astype(int) # Sometimes exclamations are useful features.
    out["tokens"] = out["tokens"].apply(lambda tokens: 
                                        [''.join([c for c in word if c not in punctuation])
                                        for word in tokens if ''.join(
                                            [c for c in word if c not in punctuation])]) # Remove punctuation.
    stop_words = set(stopwords.words("english")) - {"not", "no"} 
    out["tokens"] = out["tokens"].apply(lambda words: [w for w in words if w not in stop_words]) # Remove contractions except not and no.
    lemmatizer = WordNetLemmatizer()
    out["tokens"] = out["tokens"].apply(lambda words: [lemmatizer.lemmatize(w) for w in words]) # Apply lemmatizer.
    out["hashtags"] = out["text"].apply(lambda x: re.findall(r"#\w+", x)) # Remove hashtags from text but keep as seprate feature.
    out["cleaned_text"] = out["tokens"].apply(lambda words: " ".join(words))
    out["target"] = out["y (label) [1 = deplatform positive, 0 = not deplatformed]"]
    return out
data_preprocessed = preprocess(raw_scraper_results)
data_preprocessed.to_csv("../test-data/preprocessed.csv")
data_preprocessed.head()

,X (Post Content),"y (label) [1 = deplatform positive, 0 = not deplatformed]",text,tokens,contains_exclamation,hashtags,cleaned_text,target
0,If you want to message me on telegram my gener...,1,if you want to message me on telegram my gener...,"[want, message, telegram, general, tele, usern...",0,[],want message telegram general tele username ME...,1
1,Please feel free to send me a direct message o...,1,please feel free to send me a direct message o...,"[please, feel, free, send, direct, message, te...",0,[],please feel free send direct message telegram ...,1
2,I’m doing quite well how about you? Would you ...,1,i am doing quite well how about you? would you...,"[quite, well, would, able, send, direct, messa...",0,[],quite well would able send direct message tele...,1
3,Send me a message on Donald Trump Mega fan pa...,1,send me a message on donald trump mega fan pa...,"[send, message, donald, trump, mega, fan, page...",0,[],send message donald trump mega fan page telegr...,1
4,I am taking MFF delivery magnet and pin commis...,1,i am taking mff delivery magnet and pin commis...,"[taking, mff, delivery, magnet, pin, commissio...",1,[],taking mff delivery magnet pin commission NUM ...,1
